### os_theme_info 초기화
- UPSERT 쿼리 사용

In [14]:
import pandas as pd

os_theme_info = pd.read_excel("./os_theme_info.xlsx")
os_theme_info.head()

,etf_tkr,etf_name,theme_kr,ytd,pdf_cnt,top_stk_name,theme_class
0,AIQ,GLOBAL X Artificial Intelligence & Technology ETF,AI&빅테크,38.5,30,알파벳A,혁신기술
1,BKCH,GLOBAL X Blockchain ETF,블록체인,140.78,30,코인베이스글로벌A,혁신기술
2,BOTZ,GLOBAL X Robotics & Artificial Intelligence ETF,로보틱스&AI,39.14,30,엔비디아,혁신기술
3,BUG,GLOBAL X Cybersecurity ETF,사이버보안,16.91,30,지스케일러,혁신기술
4,CLOU,GLOBAL X Cloud Computing ETF,클라우드컴퓨팅,20.32,30,지스케일러,혁신기술


In [ ]:
from fetch_data import fetch_data_from_db

for i, row in os_theme_info.iterrows():
    etf_tkr, etf_name, theme_kr, ytd, pdf_cnt, top_stk_name, theme_class = row
    
    if ytd=="-":
        ytd="NULL"
    
    sql = f"""
    INSERT INTO os_theme_info(
        etf_tkr, etf_name,
        theme_kr,
        ytd,
        pdf_cnt,
        top_stk_name,
        theme_class
        )
    VALUES("{etf_tkr}", "{etf_name}", "{theme_kr}", {ytd}, {pdf_cnt}, "{top_stk_name}", "{theme_class}")
    ON DUPLICATE KEY UPDATE 
        etf_tkr="{etf_tkr}",
        etf_name="{etf_name}", 
        theme_kr="{theme_kr}",
        ytd={ytd},
        pdf_cnt={pdf_cnt},
        top_stk_name="{top_stk_name}", 
        theme_class="{theme_class}"
    """
    
    print(sql)
    fetch_data_from_db(query=sql)

### os_pdf_info 초기화
- UPSERT 쿼리 사용
- 대량(약 1600개)의 UPSERT 쿼리이므로 한번에 실행시켜야합니다

In [7]:
import pandas as pd

os_pdf_info = pd.read_csv("./data_before_db_insert/os_pdf_info.csv", index_col=0)
os_pdf_info.reset_index(drop=True, inplace=True)
os_pdf_info.head()

,child_stk_tkr,child_stk_name,etf_tkr
0,LLY,ELI LILLY & CO,AGNG
1,4519 JP,CHUGAI PHARMA CO,AGNG
2,ALC SW,ALCON INC,AGNG
3,WELL,WELLTOWER INC,AGNG
4,NOVOB DC,NOVO NORDISK A/S-B,AGNG


In [12]:
from fetch_data import with_db_connection

query_parameter = []

for i, row in os_pdf_info.iterrows():
    query_parameter.append((row["child_stk_tkr"], row["child_stk_name"], row["etf_tkr"]))
    
sql = f"""
INSERT INTO os_pdf_info(
    `child_stk_tkr`,
    `child_stk_name`,
    `etf_tkr`
    )
VALUES(%s, %s, %s)
ON DUPLICATE KEY UPDATE 
    `child_stk_tkr`=VALUES(`child_stk_tkr`),
    `child_stk_name`=VALUES(`child_stk_name`),
    `etf_tkr`=VALUES(`etf_tkr`)
"""

print(sql)

@with_db_connection
def upsert_once(*args, **kwargs):
    
    conn = kwargs.pop("connection")
    sql = kwargs.pop("query")
    query_parameter = kwargs.pop("query_parameter")
    
    cursor = conn.cursor()
    
    cursor.executemany(sql, query_parameter)
    
    return cursor.fetchall()
    
upsert_once(query=sql, query_parameter=query_parameter)


INSERT INTO os_pdf_info(
    `child_stk_tkr`,
    `child_stk_name`,
    `etf_tkr`
    )
VALUES(%s, %s, %s)
ON DUPLICATE KEY UPDATE 
    `child_stk_tkr`=VALUES(`child_stk_tkr`),
    `child_stk_name`=VALUES(`child_stk_name`),
    `etf_tkr`=VALUES(`etf_tkr`)



[]

### os_stk_info 초기화

In [5]:
import pandas as pd

os_stk_info = pd.read_csv("./data_before_db_insert/os_stk_info.csv", index_col=0)
os_stk_info.head()

,stk_tkr,float_shares
0,A,294401078
1,AA,177057698
2,AAON,44908066
3,AAPL,15617347029
4,ABBV,1762275557


In [13]:
from fetch_data import with_db_connection

query_parameter = []

for i, row in os_stk_info.iterrows():
    query_parameter.append((row["stk_tkr"], row["float_shares"]))
    
sql = f"""
INSERT INTO os_stk_info(
    `stk_tkr`,
    `float_shares`
    )
VALUES(%s, %s)
ON DUPLICATE KEY UPDATE 
    `stk_tkr`=VALUES(`stk_tkr`),
    `float_shares`=VALUES(`float_shares`)
"""

print(sql)

@with_db_connection
def upsert_once(*args, **kwargs):
    
    conn = kwargs.pop("connection")
    sql = kwargs.pop("query")
    query_parameter = kwargs.pop("query_parameter")
    
    cursor = conn.cursor()
    
    cursor.executemany(sql, query_parameter)
    
    return cursor.fetchall()
    
upsert_once(query=sql, query_parameter=query_parameter)


INSERT INTO os_stk_info(
    `stk_tkr`,
    `float_shares`
    )
VALUES(%s, %s)
ON DUPLICATE KEY UPDATE 
    `stk_tkr`=VALUES(`stk_tkr`),
    `float_shares`=VALUES(`float_shares`)



[]

### os_stk_price 초기화

In [3]:
import pandas as pd

os_stk_price = pd.read_csv("./data_before_db_insert/os_stk_price.csv", index_col=0)
os_stk_price.dropna(inplace=True)
os_stk_price.head()

,date,close,stk_tkr
0,2022-08-15,132.998550,A
1,2022-08-16,131.876160,A
2,2022-08-17,141.332077,A
3,2022-08-18,139.027679,A
4,2022-08-19,136.693512,A


In [4]:
os_stk_price[os_stk_price['stk_tkr']=='GOOGL']

,date,close,stk_tkr
70531,2022-08-15,122.080002,GOOGL
70532,2022-08-16,121.699997,GOOGL
70533,2022-08-17,119.550003,GOOGL
70534,2022-08-18,120.169998,GOOGL
70535,2022-08-19,117.209999,GOOGL
...,...,...,...
70777,2023-08-08,131.399994,GOOGL
70778,2023-08-09,129.660004,GOOGL
70779,2023-08-10,129.690002,GOOGL
70780,2023-08-11,129.559998,GOOGL


In [10]:
from fetch_data import with_db_connection

query_parameter = []

for i, row in os_stk_price.iterrows():
    query_parameter.append((row["stk_tkr"], row["close"], row["date"]))
    
sql = f"""
INSERT INTO os_stk_price(
    `stk_tkr`,
    `close`,
    `date`
    )
VALUES(%s, %s, %s)
ON DUPLICATE KEY UPDATE 
    `stk_tkr`=VALUES(`stk_tkr`),
    `close`=VALUES(`close`),
    `date`=VALUES(`date`)
"""

print(sql)

@with_db_connection
def upsert_once(*args, **kwargs):
    
    conn = kwargs.pop("connection")
    sql = kwargs.pop("query")
    query_parameter = kwargs.pop("query_parameter")
    
    cursor = conn.cursor()
    
    cursor.executemany(sql, query_parameter)
    
    return cursor.fetchall()
    
upsert_once(query=sql, query_parameter=query_parameter)


INSERT INTO os_stk_price(
    `stk_tkr`,
    `close`,
    `date`
    )
VALUES(%s, %s, %s)
ON DUPLICATE KEY UPDATE 
    `stk_tkr`=VALUES(`stk_tkr`),
    `close`=VALUES(`close`),
    `date`=VALUES(`date`)



[]